## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:



In [1]:
import pandas as pd
import sqlite3

# print big numbers in pd.DataFrame (instead of scientific notation)
pd.set_option('display.precision', 12)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Let's make some helper functions
- Something to convert $123,456,789.00 to float


In [3]:
def convert_money_string(money_series):
    '''
    This function takes a pd.Series with string values representing money (USD) and converts that string to float
    '''
    return money_series.str.replace('$','').str.replace(',','').astype(float)

## Find column names and info in various csv files

### What's in bom.movie_gross.csv.gz ?
- `foreign_gross` has ~1300 missing values, `domestic_gross` and `studio` have small number of missing values
- `domestic_gross` has dtype float64, but `foreign_gross` has dtype object
- Combination of `title` and `year` may serve as key